In [2]:
import pandas as pd

# 使用例
df = pd.DataFrame({
    '部店コード': ['1234', '1234', '5678', '9012', '9012', '3456'],
    '課Grコード': [None, '01', None, '01', '02', None],
    '部店名称': ['東京支店', '東京支店', '大阪支店', '名古屋支店', '名古屋支店', '福岡支店'],
    '課Gr名称': [None, '営業課', None, '営業課', '管理課', None]
})
df


,部店コード,課Grコード,部店名称,課Gr名称
0,1234,None,東京支店,None
1,1234,01,東京支店,営業課
2,5678,None,大阪支店,None
3,9012,01,名古屋支店,営業課
4,9012,02,名古屋支店,管理課
5,3456,None,福岡支店,None


In [14]:
from tabulate import tabulate
print(tabulate(df, tablefmt='grid', headers=df.columns, showindex=False))

+--------------+--------------+------------+------------+
|   部店コード |   課Grコード | 部店名称   | 課Gr名称   |
+==============+==============+============+============+
|         1234 |              | 東京支店   |            |
+--------------+--------------+------------+------------+
|         1234 |           01 | 東京支店   | 営業課     |
+--------------+--------------+------------+------------+
|         5678 |              | 大阪支店   |            |
+--------------+--------------+------------+------------+
|         9012 |           01 | 名古屋支店 | 営業課     |
+--------------+--------------+------------+------------+
|         9012 |           02 | 名古屋支店 | 管理課     |
+--------------+--------------+------------+------------+
|         3456 |              | 福岡支店   |            |
+--------------+--------------+------------+------------+


In [15]:
grouped = df.groupby('部店コード')
def check_group(group):
    has_bu = group['課Grコード'].isnull().any()  # 部明細の存在チェック
    has_ka = group['課Grコード'].notnull().any()  # 課Gr明細の存在チェック
    return pd.Series({'has_bu': has_bu, 'has_ka': has_ka})
result = grouped.apply(check_group)

In [16]:
print(tabulate(result, tablefmt='grid', headers=df.columns, showindex=False))



+--------------+--------------+
|   部店コード |   課Grコード |
+==============+==============+
|            1 |            1 |
+--------------+--------------+
|            1 |            0 |
+--------------+--------------+
|            1 |            0 |
+--------------+--------------+
|            0 |            1 |
+--------------+--------------+


In [34]:
from pprint import pprint
for name, series in grouped:
    print(name)
    print(series)

1234
  部店コード 課Grコード  部店名称 課Gr名称
0  1234   None  東京支店  None
1  1234     01  東京支店   営業課
3456
  部店コード 課Grコード  部店名称 課Gr名称
5  3456   None  福岡支店  None
5678
  部店コード 課Grコード  部店名称 課Gr名称
2  5678   None  大阪支店  None
9012
  部店コード 課Grコード   部店名称 課Gr名称
3  9012     01  名古屋支店   営業課
4  9012     02  名古屋支店   管理課


In [35]:
for name, group in grouped:
    print(f"部店コード {name} のグループ:")
    print(group)
    print(f"課Grコードがnullの行がある: {group['課Grコード'].isnull().any()}")
    print(f"課Grコードが非nullの行がある: {group['課Grコード'].notnull().any()}")
    print("\n")

部店コード 1234 のグループ:
  部店コード 課Grコード  部店名称 課Gr名称
0  1234   None  東京支店  None
1  1234     01  東京支店   営業課
課Grコードがnullの行がある: True
課Grコードが非nullの行がある: True


部店コード 3456 のグループ:
  部店コード 課Grコード  部店名称 課Gr名称
5  3456   None  福岡支店  None
課Grコードがnullの行がある: True
課Grコードが非nullの行がある: False


部店コード 5678 のグループ:
  部店コード 課Grコード  部店名称 課Gr名称
2  5678   None  大阪支店  None
課Grコードがnullの行がある: True
課Grコードが非nullの行がある: False


部店コード 9012 のグループ:
  部店コード 課Grコード   部店名称 課Gr名称
3  9012     01  名古屋支店   営業課
4  9012     02  名古屋支店   管理課
課Grコードがnullの行がある: False
課Grコードが非nullの行がある: True




In [40]:
for name, series in grouped:
    print(name)
    pprint(series['課Grコード'].isnull())
    pprint(series['課Grコード'].notnull())

1234
0     True
1    False
Name: 課Grコード, dtype: bool
0    False
1     True
Name: 課Grコード, dtype: bool
3456
5    True
Name: 課Grコード, dtype: bool
5    False
Name: 課Grコード, dtype: bool
5678
2    True
Name: 課Grコード, dtype: bool
2    False
Name: 課Grコード, dtype: bool
9012
3    False
4    False
Name: 課Grコード, dtype: bool
3    True
4    True
Name: 課Grコード, dtype: bool


In [41]:
import pandas as pd

def detect_incomplete_records(df):
    # 部店コードと課Grコードをキーにしてグループ化
    grouped = df.groupby('部店コード')

    # 各グループ内で部明細と課Gr明細の存在をチェック
    def check_group(group):
        has_bu = group['課Grコード'].isnull().any()  # 部明細の存在チェック
        has_ka = group['課Grコード'].notnull().any()  # 課Gr明細の存在チェック
        return pd.Series({'has_bu': has_bu, 'has_ka': has_ka})

    result = grouped.apply(check_group)

    # 部明細のみ、または課Gr明細のみの部店コードを抽出
    # Groupby実行結果のindexにGroupby毎の部店が設定されているのを利用する
    incomplete_bu = result[result['has_bu'] & ~result['has_ka']].index
    incomplete_ka = result[~result['has_bu'] & result['has_ka']].index

    # ただし返しているのはIndexオブジェクト
    return incomplete_bu, incomplete_ka

# 使用例
df = pd.DataFrame({
    '部店コード': ['1234', '1234', '5678', '9012', '9012', '3456'],
    '課Grコード': [None, '01', None, '01', '02', None],
    '部店名称': ['東京支店', '東京支店', '大阪支店', '名古屋支店', '名古屋支店', '福岡支店'],
    '課Gr名称': [None, '営業課', None, '営業課', '管理課', None]
})

incomplete_bu, incomplete_ka = detect_incomplete_records(df)

print("部明細のみの部店コード:", incomplete_bu.tolist())
print("課Gr明細のみの部店コード:", incomplete_ka.tolist())

部明細のみの部店コード: ['3456', '5678']
課Gr明細のみの部店コード: ['9012']


In [42]:
incomplete_bu

Index(['3456', '5678'], dtype='object', name='部店コード')

In [43]:
incomplete_ka

Index(['9012'], dtype='object', name='部店コード')

In [44]:
result

,has_bu,has_ka
部店コード,,
1234,True,True
3456,True,False
5678,True,False
9012,False,True


In [45]:
result[result['has_bu'] & ~result['has_ka']]

,has_bu,has_ka
部店コード,,
3456,True,False
5678,True,False


In [19]:
grouped.first()
print(tabulate(grouped.first(), tablefmt='grid', headers=grouped.first().columns, showindex=True))


+------+--------------+------------+------------+
|      |   課Grコード | 部店名称   | 課Gr名称   |
+======+==============+============+============+
| 1234 |           01 | 東京支店   | 営業課     |
+------+--------------+------------+------------+
| 3456 |              | 福岡支店   |            |
+------+--------------+------------+------------+
| 5678 |              | 大阪支店   |            |
+------+--------------+------------+------------+
| 9012 |           01 | 名古屋支店 | 営業課     |
+------+--------------+------------+------------+


In [48]:
grouped.last()

,課Grコード,部店名称,課Gr名称
部店コード,,,
1234,01,東京支店,営業課
3456,None,福岡支店,None
5678,None,大阪支店,None
9012,02,名古屋支店,管理課


In [49]:
grouped.groups

{'1234': [0, 1], '3456': [5], '5678': [2], '9012': [3, 4]}